In [1]:
import pandas as pd
from pandarallel import pandarallel
import ast
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
from transformers import BertTokenizer, BertModel
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer


In [2]:
from gensim.models import Word2Vec

In [3]:
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 10 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [4]:
dataRew=pd.read_csv('../Dataset/datiClean.csv')
dataMovie=pd.read_csv('../Dataset/movieclean.csv')

In [5]:
dataRew["clean_review"]=dataRew.loc[:,"clean_review"].parallel_apply(ast.literal_eval)

In [6]:
dataMovie["plot_clean"]=dataMovie.loc[:,"plot_clean"].parallel_apply(ast.literal_eval)

In [7]:
dataRew.drop(['review_text'],axis=1,inplace=True)

In [8]:
dataMovie.drop(['plot_synopsis','plot_summary'],axis=1,inplace=True)

### Split the Dataset

important distribute well the labels in the train and test dataset

In [9]:
dataRew.drop(['review_date','movie_id','user_id','rating','review_summary'],axis=1,inplace=True)

In [10]:
x=dataRew['clean_review']
y=dataRew['is_spoiler']

In [11]:
## Stratify bilancia i dataset 
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y,random_state=42)

In [12]:
y_train.value_counts()

is_spoiler
False    338391
True     120739
Name: count, dtype: int64

In [13]:
y_test.value_counts()

is_spoiler
False    84598
True     30185
Name: count, dtype: int64

### Apply Logistic Regression and Naive Bayes, with k fold

### Function for computing the result of the model

In [14]:
def print_mean():
    # Calcola le medie delle metriche
    mean_accuracy = np.mean(metrics['accuracy'])
    mean_precision = np.mean(metrics['precision'])
    mean_recall = np.mean(metrics['recall'])
    mean_f1_score = np.mean(metrics['f1_score'])

    # Stampa le medie delle metriche
    print("Mean Accuracy:", mean_accuracy)
    print("Mean Precision:", mean_precision)
    print("Mean Recall:", mean_recall)
    print("Mean F1 Score:", mean_f1_score)

In [15]:
def print_test(y_pred_test):
    # Calcolo delle metriche di valutazione sul set di test
    accuracy_test = accuracy_score(y_test, y_pred_test)
    precision_test = precision_score(y_test, y_pred_test)
    recall_test = recall_score(y_test, y_pred_test)
    f1_score_test = f1_score(y_test, y_pred_test)

    # Stampa delle metriche di valutazione sul set di test
    print("Test Accuracy:", accuracy_test)
    print("Test Precision:", precision_test)
    print("Test Recall:", recall_test)
    print("Test F1 Score:", f1_score_test)

In [16]:
metrics = {
    'accuracy': [],
    'precision': [],
    'recall': [],
    'f1_score': []
}
def computeLogistic(folds,iter,X,y_train):
    logistic_reg=LogisticRegression(max_iter=iter)
    ## Stratified k-fold grant me a balance division of classes

    kf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=42)
    for train_index, val_index in kf.split(X, y_train):
        X_fold_train, X_fold_val = X[train_index], X[val_index]
        y_fold_train, y_fold_val = y_train[train_index], y_train[val_index]
        
        # Training
        logistic_reg.fit(X_fold_train, y_fold_train)
        
        # Validation
        y_pred = logistic_reg.predict(X_fold_val)
        
        # Compute metrics
        metrics['accuracy'].append(accuracy_score(y_fold_val, y_pred))
        metrics['precision'].append(precision_score(y_fold_val, y_pred))
        metrics['recall'].append(recall_score(y_fold_val, y_pred))
        metrics['f1_score'].append(f1_score(y_fold_val, y_pred))
    return logistic_reg

In [17]:
metrics = {
    'accuracy': [],
    'precision': [],
    'recall': [],
    'f1_score': []
}
def compute_naive(folds,X,y_train):
    kf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=42)
    naive_bayes = MultinomialNB()
    
    for train_index, val_index in kf.split(X, y_train):
        X_fold_train, X_fold_val = X[train_index], X[val_index]
        y_fold_train, y_fold_val = y_train[train_index], y_train[val_index]
        
        # Training
        naive_bayes.fit(X_fold_train, y_fold_train)
        
        # Validation
        y_pred = naive_bayes.predict(X_fold_val)
        
        # Compute metrics
        metrics['accuracy'].append(accuracy_score(y_fold_val, y_pred))
        metrics['precision'].append(precision_score(y_fold_val, y_pred))
        metrics['recall'].append(recall_score(y_fold_val, y_pred))
        metrics['f1_score'].append(f1_score(y_fold_val, y_pred))
    return naive_bayes

### Using Bag of Words

first create the text, using the array of vector

In [18]:
text=[" ".join(word) for word in X_train]

In [19]:
textT=[" ".join(word) for word in X_test]

In [20]:
## Bag of Words for train
vect=CountVectorizer()
X=vect.fit_transform(text)

In [21]:
## Bag of Words for test
X_t=vect.transform(textT)

In [22]:
y_train=y_train.values
y_test=y_test.values

## Logistic Regression

### Result Train and Validation

In [23]:
logistic_reg=computeLogistic(5,1000,X,y_train)
print_mean()

c:\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_resu

Mean Accuracy: 0.7686668263890402
Mean Precision: 0.6023523766452841
Mean Recall: 0.35405298385004347
Mean F1 Score: 0.44596791661009466


### Result Test

In [24]:
y_pred_test = logistic_reg.predict(X_t)
print_test(y_pred_test)


Test Accuracy: 0.7684151834330868
Test Precision: 0.6008509208979455
Test Recall: 0.35557396057644525
Test F1 Score: 0.4467615717615718


## Naive Bayes

### Result Validation, Training

In [25]:
naive_bayes=compute_naive(5,X,y_train)
print_mean()

Mean Accuracy: 0.7532997190338249
Mean Precision: 0.5520644856460131
Mean Recall: 0.4214255696736403
Mean F1 Score: 0.4705832396362616


### Result Test

In [26]:
y_pred_test = naive_bayes.predict(X_t)

print_test(y_pred_test)

Test Accuracy: 0.7393080856921321
Test Precision: 0.50453789663295
Test Recall: 0.4825244326652311
Test F1 Score: 0.4932856925135048


## Tf-idf

In [27]:

tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(text)
X_test_tfidf = tfidf_vectorizer.transform(textT)

### Logistic Regression

In [28]:
logistic_reg=computeLogistic(5,1000,X_train_tfidf,y_train)
print_mean()

Mean Accuracy: 0.7615664408773115
Mean Precision: 0.5864302834810246
Mean Recall: 0.3908789671580638
Mean F1 Score: 0.45996195952818264


In [29]:
y_pred_test = logistic_reg.predict(X_test_tfidf)
print_test(y_pred_test)

Test Accuracy: 0.7792791615483129
Test Precision: 0.6541836215666328
Test Recall: 0.340864667881398
Test F1 Score: 0.44819550018513277


### Naive Bayes

In [30]:
naive_bayes=compute_naive(5,X_train_tfidf,y_train)
print_mean()

Mean Accuracy: 0.7564028706466578
Mean Precision: 0.6155653905621541
Mean Recall: 0.29955939422689715
Mean F1 Score: 0.35732178852560276


In [31]:
y_pred_test = naive_bayes.predict(X_test_tfidf)

print_test(y_pred_test)

Test Accuracy: 0.7404493696801792
Test Precision: 0.7405140758873929
Test Recall: 0.020043067748881895
Test F1 Score: 0.03902974001677311


### Word2Vec

In [32]:
word2vec_model = Word2Vec(X_train, vector_size=100, window=5, min_count=1, sg=1)

In [33]:
X_word2vec = np.array([np.mean([word2vec_model.wv[word] for word in text], axis=0) for text in X_train])

In [34]:
X_word2vec

array([[-0.04834579,  0.2841282 , -0.10264044, ..., -0.16725683,
        -0.11757316, -0.03393182],
       [-0.05603926,  0.15854234, -0.0811372 , ..., -0.12656689,
        -0.01731962, -0.04962974],
       [-0.08749289,  0.16103889, -0.0255836 , ..., -0.11368482,
        -0.06867209, -0.03077317],
       ...,
       [-0.01828191,  0.07131239, -0.04039155, ..., -0.17059858,
        -0.08910269, -0.04211525],
       [-0.03206557,  0.12978096, -0.03692603, ..., -0.10016799,
        -0.04412956, -0.11155412],
       [-0.04023359,  0.11168694, -0.00288379, ..., -0.1801767 ,
        -0.05142977, -0.07167739]], dtype=float32)

In [35]:
logistic_reg=computeLogistic(5,1000,X_word2vec,y_train)
print_mean()

Mean Accuracy: 0.7559188029534119
Mean Precision: 0.611770225040668
Mean Recall: 0.27949710753400653
Mean F1 Score: 0.3455985286334403


In [36]:
naive_bayes=compute_naive(5,X_train_tfidf,y_train)
print_mean()

### Apply Pre-Trained Model

In [ ]:
text